# 循环神经网络


## nn.RNNCell
因为时间序列的关系，例如天气预测，都会和上一个样本之间产生关系，
所以构建一个rnn_cell来获得上一个cell和这一次的样本进行相加求和


In [2]:
import torch.nn as nn
import torch

batch_size = 1
seq_len = 3
input_size = 4
hidden_size = 2


cell = nn.RNNCell(input_size= input_size,hidden_size=hidden_size)
#hidden = cell(input,hidden) 构建一个cell


dataset = torch.randn(seq_len,batch_size,input_size)
hidden = torch.zeros(batch_size,hidden_size)

for idx,input in enumerate(dataset):
    print('='*20,idx,"="*20)
    print('input size=',input.shape)
    
    hidden  = cell(input,hidden)
    
    print('output seze=',hidden.shape)
    
    print(hidden)

==================== 0 ====================
input size= torch.Size([1, 4])
output seze= torch.Size([1, 2])
tensor([[-0.3664, -0.6673]], grad_fn=<TanhBackward0>)
==================== 1 ====================
input size= torch.Size([1, 4])
output seze= torch.Size([1, 2])
tensor([[-0.4712,  0.6672]], grad_fn=<TanhBackward0>)
==================== 2 ====================
input size= torch.Size([1, 4])
output seze= torch.Size([1, 2])
tensor([[ 0.7873, -0.7466]], grad_fn=<TanhBackward0>)


## 使用nn.Rnn  
![rnn](1.png)

In [1]:
import torch.nn as nn
import torch
import torch.optim as optim

batch_size = 1
seq_len = 5
input_size = 4
hidden_size = 8
num_layers = 2
num_class = 4
embedding_size = 10




class Model(nn.Module):
    def __init__(self,input_size,hidden_size,batch_size,num_layers=1):
        super(Model,self).__init__()
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        ##增加一层embedding
        
        self.emb = nn.Embedding(self.input_size,embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size,
                         hidden_size=self.hidden_size,
                         num_layers=self.num_layers,
                         batch_first=True)
        
        #增加全连接层
        self.fc = nn.Linear(hidden_size,num_class)
        
    def forward(self,x):
        hidden = torch.zeros(self.num_layers,self.batch_size,self.hidden_size)
        
        ##先进行一此emb 然后进入rnn 然后linear
        x = self.emb(x)
        out,_ = self.rnn(x,hidden)
        out = self.fc(out)
        
        return out.view(-1,num_class)
    
    
net = Model(input_size,hidden_size,batch_size,num_layers)

#onehot
    
idx2char=['e','h','l','o']
#x_data=[1,0,2,2,3]
y_data=[3,1,2,3,2]
x_data=[[1,0,2,2,3]]
##这里可以使用skearn的one hot编码
#import numpy as np
#from sklearn.preprocessing import OneHotEncoder as oh
#x_data=np.array(x_data).reshape(-1,1)
#encoder=oh(sparse=False)
#y_onehot = encoder.fit_transform(x_data)

#inputs = torch.Tensor(y_onehot).view(seq_len,batch_size,input_size)


inputs = torch.LongTensor(x_data)
labels = torch.LongTensor(y_data)
        
     
        
#train
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr = 0.5)

for epoch in range(115):
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    
    _,idx = outputs.max(dim=1)
    idx = idx.data.numpy()
    print('Predict:',''.join([idx2char[x] for x in idx]),end='')
    print(" ,Epoch [%d/15] loss =%.3f "%(epoch+1,loss.item()))
    
    
        
        


Predict: oohho ,Epoch [1/15] loss =1.420 
Predict: olooo ,Epoch [2/15] loss =1.792 
Predict: lelle ,Epoch [3/15] loss =1.798 
Predict: lhlll ,Epoch [4/15] loss =0.725 
Predict: ohlol ,Epoch [5/15] loss =0.124 
Predict: ohlol ,Epoch [6/15] loss =0.097 
Predict: ohlol ,Epoch [7/15] loss =0.008 
Predict: ohlol ,Epoch [8/15] loss =0.004 
Predict: ohlol ,Epoch [9/15] loss =0.003 
Predict: ohlol ,Epoch [10/15] loss =0.002 
Predict: ohlol ,Epoch [11/15] loss =0.001 
Predict: ohlol ,Epoch [12/15] loss =0.001 
Predict: ohlol ,Epoch [13/15] loss =0.001 
Predict: ohlol ,Epoch [14/15] loss =0.001 
Predict: ohlol ,Epoch [15/15] loss =0.001 
Predict: ohlol ,Epoch [16/15] loss =0.000 
Predict: ohlol ,Epoch [17/15] loss =0.000 
Predict: ohlol ,Epoch [18/15] loss =0.000 
Predict: ohlol ,Epoch [19/15] loss =0.000 
Predict: ohlol ,Epoch [20/15] loss =0.000 
Predict: ohlol ,Epoch [21/15] loss =0.000 
Predict: ohlol ,Epoch [22/15] loss =0.000 
Predict: ohlol ,Epoch [23/15] loss =0.000 
Predict: ohlol ,Epoc

In [1]:
x_data=[1,0,2,2,3]
y_data=[3,1,2,3,2]


import numpy as np
from sklearn.preprocessing import OneHotEncoder as oh
x_data=np.array(x_data).reshape(-1,1)
encoder=oh(sparse=False)
y_onehot = encoder.fit_transform(x_data)
y_onehot

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [4]:
out

tensor([[[-0.9300, -0.4235]],

        [[-0.5681,  0.9306]],

        [[-0.1181,  0.4131]]], grad_fn=<StackBackward0>)